In [1]:
import pickle
import numpy as np
import bcolz
import openke
# from openke.config import Trainer, Tester
# from openke.module.model import TransE
# from openke.module.loss import MarginLoss
# from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader
import json
import collections
from scipy.sparse import csc_matrix
import scipy

In [2]:
train_dataloader = TrainDataLoader(
	in_path = "./benchmarks/FB60K/",
	nbatches = 100,
	threads = 8,
	sampling_mode = "normal",
	bern_flag = 1,
	filter_flag = 1,
	neg_ent = 25,
	neg_rel = 0)
entity_size = train_dataloader.get_ent_tot()

In [3]:
entity_size

69515

In [4]:
base_path = '/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/'
entity2id = {}
with open("/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/entity2id.txt") as f:
    line = f.readline().split()
    max_id = int(line[0])
    line = f.readline().split()
    while line:
        entity2id[line[0]] = int(line[1])
        line = f.readline().split()

In [5]:
with open(base_path + 'train.json') as f:
    train_file = json.load(f) 

In [59]:
similarity_matrix = collections.Counter()
for data in train_file:
#     head_id = "/" + "/".join(data['head']['id'].split("."))
    head_entity =  data['head']['id']
    tail_entity = data['tail']['id']
    head_idx = entity2id[head_entity]
    tail_idx = entity2id[tail_entity]
    if head_idx > tail_idx:
        tmp = head_idx
        head_idx = tail_idx
        tail_idx = tmp
    fin_key = str(head_idx) + '_' + str(tail_idx)                                                                                      
    similarity_matrix[fin_key] += 1
similarity_matrix = dict(similarity_matrix)

In [63]:
row = []
col = []
data = []
for k,v in similarity_matrix.items():
    k = k.split("_")
    i = int(k[0])
    j = int(k[1])
    row.append(i)
    col.append(j)
    data.append(v)

In [64]:
sim_matrix = csc_matrix((data, (row, col)), shape=(entity_size, entity_size))

In [65]:
scipy.sparse.save_npz(base_path + 'sparse_matrix.npz', sim_matrix)
sparse_matrix = scipy.sparse.load_npz(base_path + 'sparse_matrix.npz')

In [2]:
base_path = '/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/'
sim_weight = scipy.sparse.load_npz(base_path + 'sparse_matrix.npz')

In [4]:
rows, cols = sim_weight.nonzero()
head = np.load("head.npy")
tail = np.load("tail.npy")

In [48]:
c = zip(head,tail)
pos = zip(rows,cols)
c = np.array(list(c))
pos= np.array(list(pos))

In [46]:
c[np.isin(c, pos)]

array([], dtype=int64)

In [37]:
head_batch = []
tail_batch = []
head_sim = []
tail_sim = []
for item in zip(head, tail):
    val1 = item[0].tolist()
    val2 = item[1].tolist()
    if (min(val1, val2), max(val1, val2)) in pos:
        head_batch.append(val1)
        tail_batch.append(val2)
        head_sim.append(min(val1, val2))
        tail_sim.append(max(val1, val2))

(468, 13364)
(11, 20548)
(562, 7802)
(3681, 22270)
(183, 27299)
(4103, 27800)
(1017, 23010)
(87, 2506)
(745, 19250)
(2358, 27055)
(459, 22738)
(5, 21973)
(11, 32955)
(123, 25275)
(5166, 5195)
(98, 8753)
(793, 3903)
(4548, 4549)
(2153, 2154)
(3451, 20662)
(3045, 26576)
(3658, 4528)
(418, 5838)
(1804, 7510)
(4517, 18826)
(19144, 25915)
(469, 21922)
(1780, 22009)
(72, 1674)
(198, 11597)
(1569, 28221)
(340, 14188)
(4266, 25506)
(2965, 6544)
(10928, 17967)
(23, 4401)
(1780, 11346)
(3996, 18164)
(4192, 7241)
(3060, 4531)
(151, 19998)
(206, 1989)
(753, 2064)
(344, 957)
(11, 6137)
(12265, 33604)
(1337, 11210)
(25089, 25970)
(670, 15235)
(1096, 3700)
(78, 22715)
(632, 938)
(4157, 19083)
(3459, 4155)
(2099, 15395)
(11, 15530)
(2545, 3770)
(2302, 22443)
(1792, 25082)
(199, 19759)
(183, 17092)
(1464, 4473)
(348, 12379)
(568, 36898)
(1282, 1515)
(20541, 20541)
(11, 14392)
(4896, 11202)
(4029, 9784)
(809, 8752)
(8454, 11948)
(1988, 5414)
(2349, 4688)
(2263, 12712)
(8544, 11409)
(529, 1665)
(6483, 64

(9987, 29257)
(7636, 44537)
(864, 65832)
(6436, 37359)
(21760, 44711)
(10005, 10751)
(15793, 38114)
(5519, 64000)
(3576, 65953)
(3911, 12998)
(623, 29492)
(2704, 16638)
(24283, 42542)
(1162, 30535)
(1717, 26628)
(30332, 52635)
(15424, 33132)
(19308, 24917)
(31338, 68042)
(9737, 28082)
(19592, 46878)
(6749, 19931)
(2671, 53744)
(3914, 12177)
(6953, 42637)
(2005, 32013)
(8920, 48741)
(9661, 11313)
(26904, 27022)
(340, 46463)
(11452, 48086)
(2489, 22065)
(1631, 3442)
(4349, 69091)
(3131, 17917)
(18972, 45217)
(4419, 51044)
(18111, 53861)
(23128, 52879)
(6705, 18469)
(4200, 16975)
(12843, 53995)
(23027, 62374)
(174, 8955)
(29608, 68224)
(9862, 56123)
(14030, 23947)
(12473, 60813)
(62, 57993)
(19541, 49085)
(18307, 19464)
(7161, 56403)
(897, 28474)
(5844, 68969)
(9769, 57404)
(3679, 68294)
(7571, 10252)
(20452, 54873)
(7081, 31762)
(1109, 21127)
(7959, 25198)
(2703, 2789)
(9608, 31420)
(9523, 42283)
(11, 3197)
(26943, 39004)
(27197, 56500)
(4863, 27371)
(1736, 19270)
(11957, 41162)
(2942, 4

(12976, 18556)
(14441, 16397)
(23, 32541)
(1680, 56422)
(651, 13344)
(1043, 69249)
(9164, 54034)
(4788, 48230)
(29308, 51747)
(27790, 31787)
(306, 14100)
(6405, 21768)
(716, 45578)
(2032, 10549)
(4804, 30258)
(787, 48083)
(13654, 21754)
(15191, 31403)
(7743, 36184)
(1527, 38083)
(509, 3899)
(7, 6239)
(2598, 36030)
(35766, 38187)
(19478, 47900)
(16817, 23282)
(17932, 60954)
(7959, 13191)
(1147, 13081)
(14021, 17330)
(12613, 51248)
(2007, 51127)
(512, 37714)
(1552, 10893)
(784, 13868)
(2503, 14176)
(7589, 52842)
(144, 39829)
(9504, 45214)
(5394, 35936)
(8070, 9449)
(1362, 57081)
(28844, 58982)
(934, 5282)
(21110, 48628)
(12528, 26825)
(1951, 4656)
(22542, 41711)
(11016, 67606)
(20362, 49585)
(12906, 23661)
(17985, 31587)
(1357, 39472)
(1139, 33391)
(947, 64542)
(11782, 38510)
(18129, 41494)
(10041, 52080)
(10217, 17468)
(27742, 48521)
(15096, 33156)
(18639, 53029)
(13999, 21738)
(4247, 23665)
(10268, 68582)
(13313, 49618)
(14072, 29183)
(2822, 17592)
(2789, 33818)
(24090, 47335)
(16719, 

(19748, 56482)
(27917, 45629)
(31411, 67736)
(469, 57397)
(26299, 32430)
(5622, 47179)
(18674, 23715)
(3344, 52823)
(968, 11142)
(382, 41782)
(6886, 34390)
(657, 65158)
(217, 38036)
(12281, 16178)
(9844, 49122)
(35, 26694)
(2233, 8779)
(444, 67031)
(11763, 57442)
(4333, 26404)
(33828, 50120)
(15368, 39271)
(646, 49343)
(3060, 32291)
(8218, 22740)
(2760, 13275)
(3654, 53110)
(6519, 60271)
(15912, 19972)
(33278, 37609)
(13013, 53012)
(25079, 65169)
(24105, 32946)
(7079, 36341)
(25111, 26468)
(22977, 34073)
(16858, 28056)
(13516, 56657)
(6472, 64424)
(25669, 46811)
(793, 996)
(4165, 28129)
(5258, 25954)
(32213, 46891)
(28969, 56644)
(27226, 44917)
(12118, 14269)
(5537, 44215)
(13709, 48854)
(19447, 32211)
(9441, 31208)
(2654, 50003)
(1153, 52038)
(19613, 26431)
(7707, 29550)
(1886, 2970)
(10051, 15608)
(8624, 65581)
(2751, 8664)
(4313, 29241)
(20820, 33125)
(35755, 41303)
(10382, 37412)
(20864, 35879)
(2370, 8354)
(6373, 27511)
(26503, 69330)
(8415, 42739)
(31233, 67730)
(15917, 44512)
(1

(1136, 25948)
(10259, 63023)
(84, 38338)
(14393, 42353)
(10145, 35482)
(10658, 54406)
(7620, 57598)
(12374, 35089)
(2100, 43150)
(23958, 27253)
(1427, 49829)
(17876, 43795)
(3638, 9978)
(20524, 41743)
(1733, 53262)
(7016, 40139)
(1552, 51328)
(8083, 30989)
(7023, 19096)
(6824, 32061)
(12880, 20433)
(28221, 49523)
(10175, 55513)
(1470, 53311)
(1955, 55148)
(2244, 62795)
(14498, 47190)
(11, 57584)
(1816, 7759)
(12671, 16080)
(7592, 8719)
(5695, 34749)
(8757, 22240)
(4598, 51806)
(3256, 31316)
(27414, 31359)
(24614, 52262)
(2833, 3852)
(2306, 41034)
(1111, 10826)
(24627, 30124)
(23005, 44180)
(21925, 50004)
(458, 26342)
(2413, 10833)
(2253, 25168)
(19095, 19682)
(12087, 63441)
(1229, 26126)
(1116, 26554)
(3153, 56890)
(11667, 15715)
(1859, 3680)
(930, 45879)
(1223, 35810)
(3602, 12674)
(20327, 34642)
(5655, 63159)
(10852, 28852)
(5606, 15599)
(422, 49810)
(123, 11031)
(14710, 48178)
(29801, 57199)
(16574, 29064)
(1288, 2401)
(2669, 40584)
(16701, 63659)
(8368, 69444)
(2080, 21699)
(4676, 

...

(13045, 14737)
(3773, 27514)
(4445, 30959)
(371, 48713)
(13660, 23045)
(11, 39652)
(5555, 54233)
(34066, 43242)
(3295, 44155)
(14357, 62554)
(646, 37442)
(21930, 66614)
(23991, 56461)
(17252, 33178)
(8604, 26955)
(6704, 6864)
(12334, 18585)
(6122, 19101)
(34940, 60561)
(20887, 54620)
(5539, 6925)
(6521, 34711)
(3372, 9199)
(32057, 49166)
(15309, 67793)
(2460, 68080)
(25239, 32817)
(21612, 29873)
(18028, 44701)
(2982, 24920)
(1528, 27211)
(426, 4439)
(9303, 14508)
(2722, 7764)
(15839, 27590)
(889, 16173)
(11853, 28363)
(153, 1761)
(5621, 8259)
(583, 19636)
(10242, 33163)
(2089, 8195)
(3581, 21385)
(19897, 64917)
(927, 55046)
(12274, 50522)
(3079, 6888)
(11, 28724)
(5709, 10679)
(481, 30166)
(10627, 61371)
(1238, 6398)
(16527, 64675)
(6597, 36522)
(7829, 28293)
(22828, 36574)
(8415, 15885)
(36900, 64838)
(4801, 27479)
(2231, 22125)
(27328, 49775)
(11520, 57813)
(11372, 30701)
(7370, 52006)
(21622, 37073)
(1788, 14262)
(7191, 41465)
(2033, 48598)
(5990, 67009)
(8704, 43476)
(17551, 27459)

(29377, 66860)
(12802, 19275)
(9212, 16733)
(3880, 31437)
(13782, 65932)
(5229, 46773)
(7627, 14272)
(17627, 25788)
(211, 67703)
(18335, 32306)
(7695, 47818)
(24677, 28266)
(7007, 56638)
(28005, 32723)
(22591, 26878)
(17644, 22113)
(16746, 17964)
(13891, 47095)
(355, 43408)
(271, 4691)
(21465, 46551)
(11473, 48672)
(15574, 58710)
(25963, 54143)
(25100, 62265)
(6194, 59072)
(13108, 24651)
(476, 7643)
(363, 41740)
(13172, 43795)
(18564, 41246)
(5995, 65409)
(14850, 22794)
(24585, 38154)
(648, 5219)
(3018, 47009)
(11092, 38153)
(35602, 49443)
(19414, 68218)
(8444, 16331)
(27172, 33198)
(2151, 36240)
(1530, 32656)
(280, 44241)
(3772, 63696)
(13442, 63409)
(297, 67263)
(33414, 60914)
(25240, 66418)
(5767, 31424)
(22645, 26248)
(14186, 40943)
(225, 8552)
(28747, 49463)
(1622, 10333)
(5075, 9075)
(7806, 41868)
(9662, 26790)
(1404, 28242)
(16089, 33387)
(8723, 23760)
(17895, 21326)
(8592, 63502)
(30664, 39973)
(9814, 66042)
(11290, 67163)
(15168, 17491)
(21889, 23211)
(7906, 60226)
(5579, 4485

(7594, 31209)
(4077, 11923)
(3848, 45502)
(5580, 35411)
(9776, 14604)
(19710, 45541)
(747, 3432)
(11565, 16416)
(380, 13422)
(828, 39284)
(16359, 26373)
(13553, 59271)
(11435, 16178)
(6837, 8506)
(11, 53050)
(30924, 65623)
(842, 20726)
(1172, 27147)
(13687, 68820)
(142, 51455)
(7552, 8676)
(6255, 53093)
(4640, 21704)
(7739, 15764)
(2731, 29891)
(2215, 17083)
(7064, 12102)
(13976, 21449)
(7111, 40572)
(4759, 9792)
(9513, 28772)
(7334, 62313)
(17729, 63845)
(17032, 58505)
(11056, 28502)
(35588, 67325)
(4625, 13641)
(9424, 34055)
(2288, 13916)
(18703, 32276)
(12001, 49212)
(9284, 32681)
(4029, 16725)
(29209, 52523)
(2988, 52836)
(1881, 14882)
(829, 55126)
(16057, 24859)
(13196, 38041)
(17432, 43753)
(4198, 48006)
(3330, 58675)
(1754, 23344)
(17779, 29867)
(1087, 69258)
(7108, 32172)
(14475, 38882)
(25998, 35515)
(2931, 58057)
(1509, 29120)
(5996, 21504)
(11513, 50491)
(1326, 60538)
(8279, 10037)
(20670, 59156)
(2263, 58455)
(16336, 66253)
(1131, 8879)
(5480, 9751)
(2857, 60993)
(12668, 20

(919, 38195)
(18843, 34970)
(12275, 17243)
(7646, 52348)
(22321, 35925)
(18223, 60848)
(947, 37595)
(11635, 31058)
(32443, 40157)
(13484, 55168)
(31925, 42450)
(198, 48561)
(1744, 50065)
(8570, 19214)
(19781, 48935)
(2937, 4966)
(8577, 27718)
(14254, 41113)
(18480, 43931)
(687, 13049)
(3423, 14924)
(10049, 14357)
(4958, 40565)
(5492, 29760)
(608, 12001)
(26795, 61214)
(20447, 37695)
(8292, 24066)
(19946, 31292)
(10905, 53016)
(5970, 21301)
(1757, 32169)
(769, 31293)
(10783, 48866)
(3959, 20393)
(17468, 24695)
(23551, 65369)
(12887, 62807)
(11018, 54743)
(1446, 45969)
(20721, 59999)
(11551, 59748)
(1279, 31657)
(18775, 27800)
(10935, 64991)
(6570, 45924)
(2350, 12005)
(3431, 40832)
(13862, 43397)
(621, 38576)
(27404, 54406)
(23709, 51910)
(11724, 47105)
(7677, 61611)
(5473, 10259)
(9750, 43245)
(14393, 33389)
(385, 2635)
(10658, 47494)
(4293, 7620)
(3241, 12374)
(2100, 51442)
(12750, 23958)
(1427, 20777)
(6101, 17876)
(3638, 34154)
(20524, 65819)
(1733, 43940)
(7016, 56009)
(22485, 6107

(1625, 42623)
(11180, 18127)
(11362, 69293)
(6542, 29846)
(650, 18643)
(17875, 41160)
(16366, 33784)
(12993, 31834)
(9179, 21307)
(8690, 21084)
(1509, 18160)
(27027, 54501)
(7, 19473)
(24142, 53120)
(159, 3306)
(107, 57207)
(13506, 33160)
(106, 11212)
(5299, 63771)
(14642, 25670)
(6876, 53459)
(4245, 59264)
(15503, 45979)
(25637, 43826)
(628, 67924)
(8979, 9533)
(30343, 60501)
(2176, 30002)
(15211, 23179)
(34034, 35621)
(24951, 42705)
(23064, 43269)
(2732, 35968)
(2575, 9262)
(8375, 22347)
(4968, 9724)
(10227, 49843)
(849, 45318)
(7306, 39185)
(9586, 43672)
(11, 33466)
(8670, 55684)
(35838, 47111)
(3546, 66959)
(12448, 51139)
(669, 27911)
(1683, 31521)
(12761, 17831)
(4076, 13275)
(3200, 41431)
(4930, 45597)
(9191, 50369)
(16193, 20846)
(14260, 51872)
(6356, 38879)
(10768, 49020)
(21792, 23950)
(78, 46248)
(916, 69211)
(10370, 52298)
(11, 25005)
(3563, 49556)
(3788, 61221)
(16231, 56040)
(4029, 51737)
(9567, 49094)
(584, 6823)
(18557, 46554)
(6873, 10001)
(8849, 28362)
(14179, 18471)
(